In [5]:
import os
import glob
import cv2
import time
import ntpath
import random
import skimage
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage import morphology
from skimage import measure
from skimage import color
from skimage import segmentation

In [2]:
print("Get file_paths.")
        
src_dir = "D:/jupyter-notebook/LiverCancer/Image_Original/"
file_paths = []

for fold in os.listdir(src_dir):
    file_paths = file_paths + [file_path for file_path in glob.glob(src_dir+fold+"/*.jpg")]

Get file_paths.


In [20]:
dst_path = "D:/jupyter-notebook/LiverCancer/Image_Mask/"
out_csv = []

for index, file_path in enumerate(file_paths):
    
    if index >= 0:
        
        time_start=time.time()
        print(file_path)
        
        if "grade0" in file_path:
            sample_label = 0
        elif "grade1" in file_path:
            sample_label = 1
        elif "grade2" in file_path:
            sample_label = 2
        elif "grade3" in file_path:
            sample_label = 3
        elif "grade4" in file_path:
            sample_label = 4
            
        #####################################################################################
        sample_image = cv2.imread(file_path)
        image_red = sample_image[:,:,2]
        kernel = skimage.morphology.disk(20)
        kernel_2 = skimage.morphology.disk(3)

        image_closing = skimage.morphology.closing(image_red, kernel_2)
        image_erosion = skimage.morphology.erosion(image_closing, kernel)
        image_reconstruction_1 = skimage.morphology.reconstruction(image_erosion,image_closing).astype(np.uint8)
        image_dialtion = skimage.morphology.dilation(image_reconstruction_1, kernel)
        image_reconstruction_2 = skimage.morphology.reconstruction(image_dialtion, image_reconstruction_1, method='erosion').astype(np.uint8)

        image_zero = (image_reconstruction_2 - image_reconstruction_2.mean()).astype(np.int)
        image_binary = image_zero < - 10
      
        for i in range(3072):
            for j in range(4080):
                for k in range(3):
                    if image_binary[i][j]==True:
                        sample_image[i][j][k]=255
                        
        #####################################################################################              
        fold_name = "grade" + str(sample_label) + "/"
        file_name = ntpath.basename(file_path)
        image_target_path = dst_path + fold_name + file_name
        
        cv2.imwrite(image_target_path, sample_image)
        
    #####################################################################################
    time_end=time.time()
    print('totally cost',time_end-time_start)
    sys.exit(0)
        

D:/jupyter-notebook/LiverCancer/Image_Original/grade0\ImageN.01.jpg
totally cost 257.3088743686676


NameError: name 'sys' is not defined

In [6]:
sample_image = cv2.resize(sample_image, (256,256)) 